In [19]:
!pip install pandas pingouin

In [20]:
import pandas as pd
import pingouin as pg

df = pd.read_excel("human_shared_cases.xlsx")
df

,case_id,rater,gpt4ov1_a,gpt4ov1_v,gpt4ov1_f,gpt4ominiv1_a,gpt4ominiv1_v,gpt4ominiv1_f,gpt4ov2_a,gpt4ov2_v,gpt4ov2_f,gpt4ominiv2_a,gpt4ominiv2_v,gpt4ominiv2_f
0,Case 1,Judge 1,3,4,3,4,5,4,3,4,5,2,4,4
1,Case 1,Judge 2,3,3,2,1,2,2,4,4,4,4,4,4
2,Case 1,Judge 3,3,3,1,5,5,1,3,3,5,3,3,5
3,Case 1,Judge 4,2,3,2,3,4,2,5,5,4,5,5,4
4,Case 1,Judge 5,4,5,4,4,5,4,5,4,3,5,4,2
5,Case 2,Judge 1,3,5,3,2,2,3,5,5,5,2,2,4
6,Case 2,Judge 2,3,3,2,4,3,3,4,4,4,3,3,3
7,Case 2,Judge 3,5,3,1,5,5,1,5,5,5,1,1,5
8,Case 2,Judge 4,1,2,1,3,5,3,5,5,5,5,4,4
9,Case 2,Judge 5,4,5,3,4,5,4,5,4,3,5,4,2


In [21]:
df_long = df.melt(
    id_vars=['case_id', 'rater'],
    var_name='condition',
    value_name='score'
)
df_long

,case_id,rater,condition,score
0,Case 1,Judge 1,gpt4ov1_a,3
1,Case 1,Judge 2,gpt4ov1_a,3
2,Case 1,Judge 3,gpt4ov1_a,3
3,Case 1,Judge 4,gpt4ov1_a,2
4,Case 1,Judge 5,gpt4ov1_a,4
...,...,...,...,...
295,Case 5,Judge 1,gpt4ominiv2_f,5
296,Case 5,Judge 2,gpt4ominiv2_f,4
297,Case 5,Judge 3,gpt4ominiv2_f,5
298,Case 5,Judge 4,gpt4ominiv2_f,4


In [22]:
# Calcular ICC
icc = pg.intraclass_corr(
    data=df_long,
    targets='case_id',
    raters='rater',
    ratings='score'
)

# Filter ICC(2,1): two-way random, absolute agreement
icc_21 = icc[icc['Type'] == 'ICC2']

print(icc_21[['Type', 'ICC', 'CI95%', 'F', 'pval']])


   Type       ICC          CI95%         F      pval
1  ICC2  0.051493  [-0.05, 0.53]  1.641394  0.212581


In [23]:
icc_by_case = {}
for case_id, group in df_long.groupby('case_id'):
    # Calculate ICC for each case_id
    # Targets are the conditions (gpt4ov1_a, etc.)
    # Raters are the judges
    icc_result = pg.intraclass_corr(
        data=group,
        targets='condition',
        raters='rater',
        ratings='score'
    )
    icc_by_case[case_id] = icc_result[icc_result['Type'] == 'ICC2'] # Filtering for ICC2 as before

# Combine results into a single DataFrame for easier viewing
icc_all_cases_df = pd.concat(icc_by_case, names=['case_id'])

print("ICC (ICC2) for Case ID:")
display(icc_all_cases_df[['Type', 'ICC', 'CI95%', 'F', 'pval']])

ICC (ICC2) for Case ID:


,,Type,ICC,CI95%,F,pval
case_id,,,,,,
Case 1,1,ICC2,0.115451,"[-0.05, 0.44]",1.686007,0.108577
Case 2,1,ICC2,0.210637,"[0.0, 0.55]",2.294759,0.025413
Case 3,1,ICC2,0.061111,"[-0.08, 0.36]",1.366445,0.222615
Case 4,1,ICC2,0.132251,"[-0.03, 0.45]",1.901899,0.065355
Case 5,1,ICC2,0.424390,"[0.18, 0.72]",5.650146,0.000014


In [24]:
# Define the two groups of metrics
version1_metrics = [
    'gpt4ov1_a', 'gpt4ov1_v', 'gpt4ov1_f',
    'gpt4ominiv1_a', 'gpt4ominiv1_v', 'gpt4ominiv1_f'
]

version2_metrics = [
    'gpt4ov2_a', 'gpt4ov2_v', 'gpt4ov2_f',
    'gpt4ominiv2_a', 'gpt4ominiv2_v', 'gpt4ominiv2_f'
]

# Filter df_long for Version 1 metrics
df_version1 = df_long[df_long['condition'].isin(version1_metrics)].copy()

# Filter df_long for Version 2 metrics
df_version2 = df_long[df_long['condition'].isin(version2_metrics)].copy()

print("--- ICC for Version 1 Metrics ---")
icc_version1 = pg.intraclass_corr(
    data=df_version1,
    targets='case_id',
    raters='rater',
    ratings='score'
)
icc_version1_21 = icc_version1[icc_version1['Type'] == 'ICC2']
display(icc_version1_21[['Type', 'ICC', 'CI95%', 'F', 'pval']])

print("\n--- ICC for Version 2 Metrics ---")
icc_version2 = pg.intraclass_corr(
    data=df_version2,
    targets='case_id',
    raters='rater',
    ratings='score'
)
icc_version2_21 = icc_version2[icc_version2['Type'] == 'ICC2']
display(icc_version2_21[['Type', 'ICC', 'CI95%', 'F', 'pval']])

--- ICC for Version 1 Metrics ---


,Type,ICC,CI95%,F,pval
1,ICC2,-0.001874,"[-0.04, 0.27]",0.96228,0.454779



--- ICC for Version 2 Metrics ---


,Type,ICC,CI95%,F,pval
1,ICC2,0.202673,"[-0.04, 0.77]",2.662607,0.070839
